In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations

from astropy.table import Table
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord
from easyquery import Query, QueryMaker

import SAGA
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord
from SAGA.utils.distance import z2v

print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()

In [ ]:
# If you have base catalogs:
# sats = C.is_sat.filter(saga.object_catalog.load_clean_specs())

# Alternatively, use pre-compiled dataset:
sats = FitsTable("https://www.dropbox.com/sh/7qeuqkq0c591k2g/AAC0c5C7erNCp0nYOhcDv9kva/saga_sats_latest.fits?dl=1").read()

In [ ]:
sats = Query(saga.host_catalog.construct_host_query("paper2_complete"), C.faint_end_limit).filter(sats)
sats = add_skycoord(sats)

In [ ]:
X = []

for sats_this in sats.group_by('HOSTID').groups:
    if len(sats_this) < 2:
        continue

    host_this = str(sats_this['HOSTID'][0])
    host_coord = SkyCoord(sats_this['HOST_RA'][0], sats_this['HOST_DEC'][0], unit="deg")
    
    sep_deg = host_coord.position_angle(sats_this['coord']).degree
    dv = z2v(sats_this['SPEC_Z']) - sats_this['HOST_VHOST']
    
    for i, j in combinations(range(len(sats_this)), 2):
        X.append((host_this, sep_deg[i], sep_deg[j], dv[i], dv[j], 1000*sats_this["HOST_DIST"][0]*sats_this['coord'][i].separation(sats_this['coord'][j]).radian))
    
X = Table(np.array(X, dtype=np.dtype([('HOSTID', '<U12'), ('pa1', np.float), ('pa2', np.float), ('dv1', np.float), ('dv2', np.float), ('sep', np.float)])))
print(len(X))

In [ ]:
X["sign"] = np.sign(X["dv1"]) * np.sign(X["dv2"])
dpa = np.abs(X["pa1"] - X["pa2"])
X["opening_angle"] = np.where(dpa < 180, 180-dpa, dpa-180)
X["corotating"] = np.where(X["opening_angle"] < 90, X["sign"] == -1, X["sign"] == 1)

In [ ]:
basic = Query('abs(dv1) > 30', 'abs(dv2) > 30')

bins = np.linspace(0, 180, 13)
hist_co = np.histogram(Query('corotating', basic).filter(X, 'opening_angle'), bins)[0]
hist_all = np.histogram(Query(basic).filter(X, 'opening_angle'), bins)[0]

p = hist_co/hist_all
plt.errorbar((bins[1:]+bins[:-1])*0.5, p, yerr=np.sqrt(p*(1-p) / hist_all), marker='s', ls=':');


bins = np.linspace(0, 180, 19)
hist_co = np.histogram(Query('corotating', basic).filter(X, 'opening_angle'), bins)[0]
hist_all = np.histogram(Query(basic).filter(X, 'opening_angle'), bins)[0]

p = hist_co/hist_all
plt.errorbar((bins[1:]+bins[:-1])*0.5, p, marker='', ls='-', alpha=0.2, c='C0');

plt.xlim(0, 180.0);
plt.axhline(0.5, c='grey', ls='--');
plt.xlabel('Opening angle [deg]');
plt.ylabel('Co-rotating Fraction')
plt.ylim(-0.05, 1.05)
plt.tight_layout()
plt.savefig('/home/yymao/Downloads/sat_coroating_pairs.pdf', bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(ncols=3, gridspec_kw={'wspace':0.1, 'width_ratios':(30,1,1)})
t = Query('corotating', basic, 'abs(dv1-dv2) > 30').filter(X) 
CS = ax[0].scatter(t["opening_angle"], t["sep"], c=np.abs(t["dv1"]-t["dv2"]), vmin=0, vmax=400, cmap="Blues_r", label="Co-rotating pair")
plt.colorbar(CS, ax[1])

t = Query('corotating == 0', basic, 'abs(dv1-dv2) > 30').filter(X)
CS = ax[0].scatter(t["opening_angle"], t["sep"], c=np.abs(t["dv1"]-t["dv2"]), vmin=0, vmax=400, cmap="Oranges_r", label="Counter-rotating pair")
cbar = plt.colorbar(CS, ax[2])
ax[1].set_yticklabels([])
ax[2].set_xlabel(r"$|\Delta v|$ [km s$^{-1}$]", fontsize='small')

lgnd = ax[0].legend(markerfirst=False, scatterpoints=3, frameon=True)
lgnd.legendHandles[0].set_color(plt.cm.Blues_r([0.2, 0.5, 0.7]))
lgnd.legendHandles[1].set_color(plt.cm.Oranges_r([0.2, 0.5, 0.7]))
ax[0].axvline(90, color="grey", ls='--')
ax[0].set_xlim(-5, 185.0);
ax[0].set_xlabel('Opening angle [deg]');
ax[0].set_ylabel('Projected separation [kpc]')
ax[0].set_ylim(0, 600.0);

In [ ]:
hist_all

In [ ]:
hist_co

In [ ]:
t = Query('corotating == 1', basic).filter(X)
np.abs(t["dv1"]-t["dv2"]).max()